In [1]:
import numpy as np
import math
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import *

from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [3]:
features = pd.read_csv('doc2vec_train(1).csv')
b = pd.read_csv('reddit_train.csv')
b = pd.factorize(b['subreddits'].astype('category'))
labels = b[0]

In [2]:
df = pd.read_csv('reddit_train.csv')
df2 = pd.read_csv('reddit_test.csv')
# df = df.sample(5000, random_state=1).copy()
df.head()

,id,comments,subreddits
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey
1,1,Ah yes way could have been :( remember when he...,nba
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends
3,3,He wouldn't have been a bad signing if we woul...,soccer
4,4,Easy. You use the piss and dry technique. Let ...,funny


In [3]:
df['category_id'] = df['subreddits'].factorize()[0]

In [4]:
tfidf = TfidfVectorizer()

# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
#                         ngram_range=(1, 2), 
#                         stop_words='english')

# tfidf = CountVectorizer(binary=True, min_df=5, ngram_range=(1, 2), stop_words='english')

# We transform each complaint into a vector
features = tfidf.fit_transform(df.comments)
features_test = tfidf.transform(df2.comments)

labels = df.subreddits

print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

Each of the 70000 complaints is represented by 74265 features (TF-IDF score of unigrams and bigrams)


In [6]:
X = df['comments'] # Collection of documents
y = df['category_id'] # Target or the labels we want to predict (i.e., the 13 different complaints of products)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

In [8]:
labels

array([ 0,  1,  2, ..., 17,  6,  9])

In [5]:
models = [
#     RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
#     LinearSVC(),
#     MultinomialNB(),
    LogisticRegression(random_state=0),
#     GaussianNB(),
#     BernoulliNB(),
#     MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(100, 20))
]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    print(model_name)
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

LogisticRegression


/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/network/home/penmetss/fastai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [6]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LogisticRegression,0.540829,0.002513


In [6]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
RandomForestClassifier,0.149357,0.003382


In [29]:
test_preds = pd.DataFrame()

In [32]:
test_preds.head()

,Id,Category
0,0,baseball
1,1,europe
2,2,anime
3,3,worldnews
4,4,trees


In [33]:
test_preds.to_csv("test.csv", index=False)

In [34]:
%cat test.csv 

Id,Category
0,baseball
1,europe
2,anime
3,worldnews
4,trees
5,Overwatch
6,baseball
7,anime
8,Music
9,europe
10,funny
11,gameofthrones
12,nfl
13,canada
14,canada
15,canada
16,Music
17,leagueoflegends
18,GlobalOffensive
19,conspiracy
20,movies
21,conspiracy
22,gameofthrones
23,baseball
24,wow
25,AskReddit
26,Overwatch
27,gameofthrones
28,trees
29,trees
30,nfl
31,GlobalOffensive
32,nba
33,conspiracy
34,gameofthrones
35,gameofthrones
36,GlobalOffensive
37,GlobalOffensive
38,nba
39,gameofthrones
40,hockey
41,gameofthrones
42,europe
43,nfl
44,gameofthrones
45,canada
46,canada
47,Overwatch
48,gameofthrones
49,AskReddit
50,leagueoflegends
51,anime
52,AskReddit
53,movies
54,Overwatch
55,canada
56,canada
57,conspiracy
58,europe
59,nba
60,trees
61,conspiracy
62,Music
63,GlobalOffensive
64,canada
65,Overwatch
66,nba
67,europe
68,AskReddit
69,GlobalOffensive
70,leagueoflegends
71,conspiracy
72,anime
73,trees
74,canada
75,movies
76,europe
77,hockey
78,anime
79,anime
80,worldnews
81,europe
82,Overwat

In [31]:
test_preds['Id'] = df2['id']
test_preds['Category'] = y_pred

In [25]:
numpy.savetxt("test.csv")

(30000,)

In [23]:
gNB = MultinomialNB(0.2)
gNB.fit(features,labels)
y_pred = gNB.predict(features_test)

In [17]:
for alpha in [0.01, 0.05, 0.07, 0.1, 0.2, 0.3, 0.4, 1]:
    gNB = MultinomialNB(alpha)
    gNB.fit(features[:50000,:],labels[:50000])
    print(alpha, np.mean(gNB.predict(features[50000:,:])==labels[50000:]))

0.01 0.54785
0.05 0.5598
0.07 0.5619
0.1 0.564
0.2 0.566
0.3 0.56515
0.4 0.5611
1 0.54905


In [ ]:
# Importing torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class LinearDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        self.x_data = features
        self.y_data = labels

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return features.shape[0]

In [ ]:

# Defining a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3818, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, 20)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return F.softmax(x) 
    
net = Net()
# print(net, net.parameters)

In [ ]:
dataset = LinearDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [ ]:

# Using similar criteria as pervious
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

In [ ]:
for epoch in range(20):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, target = data
        
        optimizer.zero_grad()
        output = net(inputs)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(data), len(train_loader.dataset),
                100. * i / len(train_loader), loss.data))